In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, accuracy_score, roc_curve, roc_auc_score
from sklearn.neural_network import MLPClassifier, MLPRegressor

In [2]:
xrd_data_raw = pd.read_csv("cod_xrd_42k.csv")

In [3]:
not_xrd = ['formula', 'spacegroup_number', 'cod_id', 'bravais_lattice']
xrd = list(xrd_data_raw.columns)
for i in not_xrd: xrd.remove(i) 

### Initial model testing zone

In [4]:
train_subset_small = xrd_data_raw.sample(n=10000)
small_X = train_subset_small[xrd]
small_Y = train_subset_small['bravais_lattice']
small_X_train, small_X_test, small_Y_train, small_Y_test = train_test_split(small_X, small_Y, test_size=0.2)

In [5]:
small_tree = DecisionTreeClassifier(criterion='entropy', random_state = 42, max_depth = 10)
small_tree.fit(small_X_train, small_Y_train)
train = small_tree.score(small_X_train, small_Y_train)
test = small_tree.score(small_X_test, small_Y_test)
print('train accuracy:', train, "\ntest accuracy:", test) 

train accuracy: 0.609125 
test accuracy: 0.335


Try ensemble learning:

In [6]:
hgbt_small = HistGradientBoostingClassifier()
hgbt_small.fit(small_X_train, small_Y_train)
train = hgbt_small.score(small_X_train, small_Y_train)
test = hgbt_small.score(small_X_test, small_Y_test)
print('train accuracy:', train, "\ntest accuracy:", test)

train accuracy: 1.0 
test accuracy: 0.4655


Nice, solid initial accuracy. Let's try grouping the bravais lattices as crystal structures first and then classifying.

In [52]:
def xtal_from_bravais(bravais):
    return bravais[0]

In [53]:
test = 'tP'

In [54]:
crystal_structure = xrd_data_raw['bravais_lattice'].apply(xtal_from_bravais)
xrd_data_raw['crystal_structure'] = crystal_structure

In [55]:
xrd_data_raw

,formula,spacegroup_number,cod_id,bravais_lattice,y0,y1,y2,y3,y4,y5,...,y171,y172,y173,y174,y175,y176,y177,y178,y179,crystal_structure
0,H26C25N4ClO3F,2,2227539,aP,4.909847e-137,2.993258e-111,3.522734e-88,8.003409e-68,3.510182e-50,2.971966e-35,...,0.034702,2.624012e-02,0.018113,0.013302,0.019432,0.030183,1.792260e-02,2.285690e-02,6.056704e-03,a
1,CoP3H36C23,2,4085709,aP,3.391626e-189,1.235121e-158,8.683022e-131,1.178397e-105,3.087251e-83,1.561391e-63,...,0.013948,1.540255e-02,0.013787,0.022637,0.021859,0.020878,2.819485e-02,2.273572e-02,1.407936e-02,a
2,H34C34S2(N2O)3,2,2210818,aP,6.144995e-218,5.954486e-185,1.113850e-154,4.022243e-127,2.803945e-102,3.773377e-80,...,0.012785,9.240285e-03,0.013426,0.016862,0.025352,0.009457,8.136539e-03,6.792385e-03,4.124101e-03,a
3,P2H44Ru2C56N14Cl4O9.25F12,2,4312692,aP,4.146075e-83,1.397867e-63,9.098168e-47,1.143146e-32,2.772737e-21,1.298301e-12,...,0.009492,1.115840e-02,0.009486,0.009029,0.013752,0.011607,9.644470e-03,7.357005e-03,3.794003e-03,a
4,H22C23SN2O3,2,2208064,aP,1.760686e-259,2.249972e-223,5.550495e-190,2.643296e-159,2.430074e-131,4.312733e-106,...,0.023459,2.670221e-02,0.018699,0.016123,0.020445,0.021638,2.263004e-02,2.730744e-02,2.011791e-02,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41809,Tl6SI4,128,4001792,tP,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.023300,1.362370e-02,0.006203,0.036890,0.013649,0.014576,6.339789e-03,3.360845e-03,7.636327e-06,t
41810,Tl6SeI4,128,4106340,tP,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.018503,9.520981e-03,0.013425,0.020926,0.005216,0.014517,2.479353e-03,2.820722e-04,3.640851e-04,t
41811,TlAuF6,92,1510149,tP,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.010852,1.153000e-01,0.070480,0.022257,0.057224,0.017475,3.080045e-02,1.403895e-02,2.332725e-02,t
41812,TlBO2,76,1511272,tP,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.023783,7.696171e-04,0.046177,0.036794,0.007691,0.034373,1.713273e-02,3.277940e-02,3.104092e-02,t


### Let's try classifying by crystal structure first

In [64]:
train_subset_small = xrd_data_raw.sample(n=10000)
X = train_subset_small[xrd]
Y = train_subset_small['crystal_structure']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [57]:
small_tree_xtal = DecisionTreeClassifier(criterion='entropy', random_state = 42, max_depth = 10)
small_tree_xtal.fit(X_train, Y_train)
train = small_tree_xtal.score(X_train, Y_train)
test = small_tree_xtal.score(X_test, Y_test)
print('train accuracy:', train, "\ntest accuracy:", test) 

train accuracy: 0.709875 
test accuracy: 0.454


In [65]:
hgbt_small = HistGradientBoostingClassifier()
hgbt_small.fit(X_train, Y_train)
train = hgbt_small.score(X_train, Y_train)
test = hgbt_small.score(X_test, Y_test)
print('train accuracy:', train, "\ntest accuracy:", test)

train accuracy: 1.0 
test accuracy: 0.5765


In [66]:
train_subset_large = xrd_data_raw.sample(n=30000)
X = train_subset_large[xrd]
Y = train_subset_large['crystal_structure']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
hgbt_large = HistGradientBoostingClassifier()
hgbt_large.fit(X_train, Y_train)
train = hgbt_large.score(X_train, Y_train)
test = hgbt_large.score(X_test, Y_test)
print('train accuracy:', train, "\ntest accuracy:", test)

train accuracy: 0.9194583333333334 
test accuracy: 0.6176666666666667


In [58]:
predictions = small_tree_xtal.predict(X_test)

In [59]:
test_subset = xrd_data_raw.sample(n=10000)

In [60]:
test_subset['predicted_crystal_structure'] = small_tree_xtal.predict(test_subset[xrd])

In [61]:
from sklearn import metrics
metrics.accuracy_score(test_subset['crystal_structure'], test_subset['predicted_crystal_structure'])

0.4869

In [62]:
xrdp = xrd + ['predicted_crystal_structure']
xrdp

['y0',
 'y1',
 'y2',
 'y3',
 'y4',
 'y5',
 'y6',
 'y7',
 'y8',
 'y9',
 'y10',
 'y11',
 'y12',
 'y13',
 'y14',
 'y15',
 'y16',
 'y17',
 'y18',
 'y19',
 'y20',
 'y21',
 'y22',
 'y23',
 'y24',
 'y25',
 'y26',
 'y27',
 'y28',
 'y29',
 'y30',
 'y31',
 'y32',
 'y33',
 'y34',
 'y35',
 'y36',
 'y37',
 'y38',
 'y39',
 'y40',
 'y41',
 'y42',
 'y43',
 'y44',
 'y45',
 'y46',
 'y47',
 'y48',
 'y49',
 'y50',
 'y51',
 'y52',
 'y53',
 'y54',
 'y55',
 'y56',
 'y57',
 'y58',
 'y59',
 'y60',
 'y61',
 'y62',
 'y63',
 'y64',
 'y65',
 'y66',
 'y67',
 'y68',
 'y69',
 'y70',
 'y71',
 'y72',
 'y73',
 'y74',
 'y75',
 'y76',
 'y77',
 'y78',
 'y79',
 'y80',
 'y81',
 'y82',
 'y83',
 'y84',
 'y85',
 'y86',
 'y87',
 'y88',
 'y89',
 'y90',
 'y91',
 'y92',
 'y93',
 'y94',
 'y95',
 'y96',
 'y97',
 'y98',
 'y99',
 'y100',
 'y101',
 'y102',
 'y103',
 'y104',
 'y105',
 'y106',
 'y107',
 'y108',
 'y109',
 'y110',
 'y111',
 'y112',
 'y113',
 'y114',
 'y115',
 'y116',
 'y117',
 'y118',
 'y119',
 'y120',
 'y121',
 'y122',
 'y

In [20]:
xrdp

In [19]:
test_subset['predicted_crystal_structure'].value_counts()

m    2385
h    1862
c    1653
o    1580
t    1429
a    1091
Name: predicted_crystal_structure, dtype: int64

In [73]:
counts = {}
accuracies = {}
for structure in test_subset['predicted_crystal_structure'].value_counts().index:
    print(structure)
    predictions = test_subset[test_subset['predicted_crystal_structure'] == structure]
    counts[structure] = (predictions['crystal_structure'].value_counts())
    accuracies[structure] = metrics.accuracy_score(predictions['crystal_structure'], predictions['predicted_crystal_structure'])

m
h
o
c
t
a


In [74]:
accuracies

{'m': 0.4240177909562639,
 'h': 0.49545697487974344,
 'o': 0.33524684270952926,
 'c': 0.7327586206896551,
 't': 0.4532476802284083,
 'a': 0.5858433734939759}

In [75]:
counts

{'m': m    1144
 o     671
 a     475
 t     233
 h     153
 c      22
 Name: crystal_structure, dtype: int64,
 'h': h    927
 t    373
 o    235
 c    212
 m    115
 a      9
 Name: crystal_structure, dtype: int64,
 'o': o    584
 m    387
 h    300
 t    293
 a     91
 c     87
 Name: crystal_structure, dtype: int64,
 'c': c    1190
 h     210
 t     144
 o      70
 m       8
 a       2
 Name: crystal_structure, dtype: int64,
 't': t    635
 h    310
 o    196
 c    167
 m     72
 a     21
 Name: crystal_structure, dtype: int64,
 'a': a    389
 m    176
 o     58
 h     22
 t     18
 c      1
 Name: crystal_structure, dtype: int64}

In [90]:
y = (counts['h'] + counts['t'] + counts['c']).sort_values()
y

a      32
m     195
o     501
t    1152
h    1447
c    1569
Name: crystal_structure, dtype: int64

In [86]:
x = (counts['o'] + counts['m'] + counts['a']).sort_values()
x

c     110
h     475
t     544
a     955
o    1313
m    1707
Name: crystal_structure, dtype: int64

In [87]:
x['c'] + x['h'] + x['t']

1129

In [88]:
x['a'] + x['m'] + x['o']

3975

In [89]:
(x['a'] + x['m'] + x['o']) / (x['a'] + x['m'] + x['o'] + x['c'] + x['h'] + x['t'])

0.7788009404388715

In [92]:
(y['c'] + y['t'] + y['h']) / (y['a'] + y['m'] + y['o'] + y['c'] + y['h'] + y['t'])

0.8513071895424836

hit it wit dat random forest

In [ ]:
train_subset_large = xrd_data_raw.sample(n=40000)
X = train_subset_large[xrd]
Y = train_subset_large['bravais_lattice']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
rf_improved = RandomForestClassifier(n_estimators= 2000, min_samples_split= 5, min_samples_leaf= 1, 
                                     max_features= 'sqrt', max_depth= 70, bootstrap= False)
rf_improved.fit(X_train, Y_train)
train = rf_improved.score(X_train, Y_train)
test = rf_improved.score(X_test, Y_test)
print('train accuracy:', train, "\ntest accuracy:", test)

In [ ]:
test = pd.read_csv('test.csv')
test['output'] = rf_improved.predict(test[xrd])
final_out = pd.DataFrame()
final_out['bravais_lattice'] = test['output']
final_out.to_csv('results_RF.csv', index_label='id')

In [ ]:
final_out